In [ ]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
import d2lzh_pytorch as d2l
import time
import ipdb
sys.path.append('..')

In [ ]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs)
)

for param in net.parameters():
    print(param.shape, param)
    init.normal_(param, mean=0, std=0.01)
    

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = .0, 0
    with torch.no_grad():
        for X, y in data_iter:
            net.eval()
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            net.train()
            n += y.shape[0]
        net.eval()
    return acc_sum / n

In [ ]:
# train model
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
num_epochs = 5
n = 0

for e in range(num_epochs):
    train_l_loss, train_acc_sum, n = 0, 0, 0
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y).sum()
        # clear grad 
        optimizer.zero_grad()
        # back-prop
        l.backward()
        train_l_loss += l.item()
        train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
        n += y.shape[0]
        ipdb.set_trace()
    print('epoch %d, loss: %.4f train acc: %.4f test acc: %.4f' % (e + 1, train_l_loss / n, train_acc_sum / n, evaluate_accuracy(test_iter, net)))

